In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
import folium

### parameters

#### data urls

In [3]:
stockholm_2019_url = '/mnt/virtio-bbc6cf3a-042b-4410-9/luftdaten/luftdaten_daten/uncleaned_ld_daten/historic_sds011/exported_data/ld_stkhlm_data_20200510_exp.csv'
# stockholm_2018_url = '/mnt/virtio-bbc6cf3a-042b-4410-9/luftdaten/luftdaten_daten/uncleaned_ld_daten/historic_sds011/exported_data/ld_stkhlm_data_2018_ONLY__20200514_exp.csv'

curr_in_data_url = stockholm_2019_url

In [4]:
# uptime url 
uptime_path = "/mnt/virtio-bbc6cf3a-042b-4410-9/luftdaten/luftdaten_daten/uncleaned_ld_daten/historic_sds011/exported_data/uptime_url.csv"

In [5]:
#### variable(s) we're working on 
working_on_col_name = 'p2'

#### input / output tables 

In [6]:
df_sensorData = -1 # this is where we load the input data

In [7]:
# this where we insert the same-length data sets for the sensors 
# - useful for holding sensor data 
df_sensorids_columns_w_sensor_readings_2pt5_mins__mean = -1
# median version
df_sensorids_columns_w_sensor_readings_2pt5_mins__median = -1

# as above but in 60 mins - using averages 
df_sensorids_columns_w_sensor_readings_60_mins__mean = -1 

# as above but in 60 mins - using median values 
df_sensorids_columns_w_sensor_readings_60_mins__median = -1 

In [8]:
# for v2 : daily median values and 90th percentile values for each sensor

# daily qualifying index
# ( ie >= 75% of each hour filled, >= 75% of the time )
df_daily_uptime_index = -1 

# where the daily median values end up 
df_daily_median_values__sensor_readings = -1

# percentile values per sensor
df_per_sensor_90th_percentile_values = -1 

In [9]:
# holds uptime statistics per sensor and time
# - same structure and order as above table, 
df_sensorids_columns_w_sensor_uptime_info_2pt5_mins = -1 

# as above but in hour frequency 
df_sensorids_columns_w_sensor_uptime_info_60_mins = -1 

In [10]:
# unique sensor ids 
unique_sensor_ids = -1 

# selected sensors?
selected_sensors = -1 

In [11]:
### secondary vars

# lat_lon_of_sensors
sensor_lat_lon = -1 

# selected sensors?
selected_sensors = -1

#### timing 

In [12]:
## the time period we're working with here 
# start_timestamp = '2018-01-01 00:00:00'
# end_timestamp = '2019-01-01 00:00:00'

start_timestamp = '2019-01-01 00:00:00'
end_timestamp = '2020-01-01 00:00:00'


# possible sample freuencies 
sensor_uptime_sampling_freq = "150S"
output_table_final_freq = "3600S"

# how many sensor periods are there in an hour?
sensor_periods_in_hour = 24


In [13]:
# relevant time ranges 
time_date_period_as_date_range__2pt5_mins = -1 
time_date_period_as_date_range__60_mins = -1 

In [14]:
# blank data rows, with the beginning and end dates 

# - good for equalling out the gathered sensor data time series 
in_data__START_TIME__blank_row = pd.DataFrame( data={ "p1" : np.NaN, "p2" : np.NaN }, index=pd.DatetimeIndex( [start_timestamp] ) )

in_data__END_TIME__blank_row = pd.DataFrame( data={ "p1" : np.NaN, "p2" : np.NaN }, index=pd.DatetimeIndex( [end_timestamp] ) )

### run! 

#### setup in/out data frames to the right length

In [15]:
# fetch relevant time-date ranges - 2.5 mins 
time_date_period_as_date_range__2pt5_mins = pd.date_range( start=start_timestamp, end=end_timestamp, freq=sensor_uptime_sampling_freq )
time_date_period_as_date_range__2pt5_mins[:10]

DatetimeIndex(['2019-01-01 00:00:00', '2019-01-01 00:02:30',
               '2019-01-01 00:05:00', '2019-01-01 00:07:30',
               '2019-01-01 00:10:00', '2019-01-01 00:12:30',
               '2019-01-01 00:15:00', '2019-01-01 00:17:30',
               '2019-01-01 00:20:00', '2019-01-01 00:22:30'],
              dtype='datetime64[ns]', freq='150S')

In [16]:
time_date_period_as_date_range__2pt5_mins.shape

(210241,)

In [17]:
df_sensorids_columns_w_sensor_readings_2pt5_mins__mean

-1

In [18]:
# date range for 60 min sampling rates
time_date_period_as_date_range__60_mins = pd.date_range( start=start_timestamp, end=end_timestamp, freq=output_table_final_freq )
time_date_period_as_date_range__60_mins[:10], time_date_period_as_date_range__60_mins[10:]

(DatetimeIndex(['2019-01-01 00:00:00', '2019-01-01 01:00:00',
                '2019-01-01 02:00:00', '2019-01-01 03:00:00',
                '2019-01-01 04:00:00', '2019-01-01 05:00:00',
                '2019-01-01 06:00:00', '2019-01-01 07:00:00',
                '2019-01-01 08:00:00', '2019-01-01 09:00:00'],
               dtype='datetime64[ns]', freq='3600S'),
 DatetimeIndex(['2019-01-01 10:00:00', '2019-01-01 11:00:00',
                '2019-01-01 12:00:00', '2019-01-01 13:00:00',
                '2019-01-01 14:00:00', '2019-01-01 15:00:00',
                '2019-01-01 16:00:00', '2019-01-01 17:00:00',
                '2019-01-01 18:00:00', '2019-01-01 19:00:00',
                ...
                '2019-12-31 15:00:00', '2019-12-31 16:00:00',
                '2019-12-31 17:00:00', '2019-12-31 18:00:00',
                '2019-12-31 19:00:00', '2019-12-31 20:00:00',
                '2019-12-31 21:00:00', '2019-12-31 22:00:00',
                '2019-12-31 23:00:00', '2020-01-01 00:00:

#### now to make those tables… to hold the various sensor and uptime data 

In [19]:
# sensor readings at 2.5 mins 
df_sensorids_columns_w_sensor_readings_2pt5_mins__mean = pd.DataFrame( index=pd.DatetimeIndex( time_date_period_as_date_range__2pt5_mins ) )
df_sensorids_columns_w_sensor_readings_2pt5_mins__median = pd.DataFrame( index=pd.DatetimeIndex( time_date_period_as_date_range__2pt5_mins ) )

# sensor readings at 60 mins 
df_sensorids_columns_w_sensor_readings_60_mins__mean = pd.DataFrame( index=pd.DatetimeIndex( time_date_period_as_date_range__2pt5_mins ) )
df_sensorids_columns_w_sensor_readings_60_mins__median = pd.DataFrame( index=pd.DatetimeIndex( time_date_period_as_date_range__2pt5_mins ) )

In [20]:
# test that the index works 
## df_sensorids_columns_w_sensor_readings_60_mins__median[ df_sensorids_columns_w_sensor_readings_60_mins__median.index > '2018-12-01 00:00:00' ]

In [21]:
# just double-check the type … so we don't have a Series, by accident
## type( df_sensorids_columns_w_sensor_readings_2pt5_mins__mean )

In [22]:
# df_sensorids_columns_w_sensor_readings_2pt5_mins__mean['test'] = 0
## df_sensorids_columns_w_sensor_readings_2pt5_mins__mean

In [23]:
## df_sensorids_columns_w_sensor_readings_2pt5_mins__mean[ df_sensorids_columns_w_sensor_readings_2pt5_mins__mean.index > '2018-12-01' ]

#### now do the uptime tables 

In [24]:
#### setup data frames for sensor data and sensor uptime data - 
##### - basically same length as the data period mentioned above 

In [25]:
df_sensorids_columns_w_sensor_uptime_info_2pt5_mins = pd.DataFrame( index=pd.DatetimeIndex( time_date_period_as_date_range__2pt5_mins ) )

# as above but in hour frequency 
df_sensorids_columns_w_sensor_uptime_info_60_mins = pd.DataFrame( index=pd.DatetimeIndex( time_date_period_as_date_range__60_mins ) )

In [26]:
# test 
## df_sensorids_columns_w_sensor_uptime_info_2pt5_mins

In [27]:
# test 
## df_sensorids_columns_w_sensor_uptime_info_60_mins

#### load data 

In [28]:
# load ld data 

# %time df_sensorData = pd.read_csv( stockholm_2019_url )

%time df_sensorData = pd.read_csv( curr_in_data_url )
curr_in_data_url

CPU times: user 7.09 s, sys: 2.73 s, total: 9.82 s
Wall time: 9.82 s


'/mnt/virtio-bbc6cf3a-042b-4410-9/luftdaten/luftdaten_daten/uncleaned_ld_daten/historic_sds011/exported_data/ld_stkhlm_data_20200510_exp.csv'

In [29]:
# check columns 
df_sensorData.columns

Index(['sensor_id', 'lat', 'lon', 'timestamp', 'p1', 'p2', 'fixed_p1_value',
       'fixed_p2_value', 'fixed_latlon_values', 'humidity__local_dht22',
       'humidity__local_bme280', 'humidity__smhi', 'humidity__met_no'],
      dtype='object')

In [30]:
# reduce columns for efficiency 
%time df_sensorData = df_sensorData[ [ 'sensor_id', 'timestamp', 'p1', 'p2' ] ]

CPU times: user 219 ms, sys: 512 µs, total: 220 ms
Wall time: 218 ms


In [31]:
# check size 
df_sensorData.shape

(7629460, 4)

In [32]:
# check column types 
df_sensorData.dtypes

sensor_id      int64
timestamp     object
p1           float64
p2           float64
dtype: object

In [33]:
# make sure the timestamp column is a datetime column
%time df_sensorData.timestamp = pd.to_datetime( df_sensorData.timestamp )

CPU times: user 1.35 s, sys: 27.3 ms, total: 1.38 s
Wall time: 1.37 s


In [34]:
# make sure the index is time(stamp)
df_sensorData = df_sensorData.set_index( 'timestamp' )

In [35]:
# reduce in-data size to relevant time period 
df_sensorData = df_sensorData[ ( df_sensorData.index > start_timestamp ) & ( df_sensorData.index < end_timestamp ) ]
df_sensorData.shape

(5829822, 3)

In [36]:
# quick check of dataset content 

# - beginning 
df_sensorData.head()

,sensor_id,p1,p2
timestamp,,,
2019-01-01 00:00:15,14296,27.07,19.23
2019-01-01 00:00:33,17650,8.60,6.03
2019-01-01 00:00:35,19597,22.33,6.63
2019-01-01 00:00:47,16147,56.07,44.27
2019-01-01 00:00:50,12673,51.03,33.63


In [37]:
# end of dataset
df_sensorData.tail()

,sensor_id,p1,p2
timestamp,,,
2019-12-31 23:59:41,36089,7.78,2.40
2019-12-31 23:59:45,12129,3.20,1.33
2019-12-31 23:59:51,30940,8.93,2.05
2019-12-31 23:59:58,7597,7.23,5.20
2019-12-31 23:59:59,22082,2.07,1.33



#### sensor_ids - get unique sensor ids 

In [38]:
# get unique sensor_ids 
all_sensor_ids = df_sensorData.sensor_id.unique()
all_sensor_ids.shape

(50,)

In [39]:
# - select sensor_ids to use.
# -- this is where it is is possible to 
#     reduce the count of sensors used
# selectd_sensor_ids = pd.Series( [19597, 7426, 20265, 10727 ] ) 
selectd_sensor_ids = all_sensor_ids
selectd_sensor_ids

array([14296, 17650, 19597, 16147, 12673, 12137, 18820, 10723, 10823,
       12129, 14837, 14133,  7597,  9436, 12393, 12141, 14300,  9344,
       18830,  7426, 12395, 18832, 18828, 10727, 12683, 19778, 19788,
        7695, 20259, 20265, 20255, 20257, 22082, 22074, 11372, 22076,
       23858, 25657, 25862, 26131, 26423, 27221, 27734, 28028, 13485,
       30940, 31846, 31848, 35747, 36089])

In [40]:
selectd_sensor_ids.shape

(50,)

### loop : assemble df of same length sensor_id sensor values - using hourly averages

#### - the sensors might have data at the beginning and end of the selected time period, but to work with it, it's good the time length is the same. 
####.  - so we're going through the sensor values and making same length time sensor data series into a table

In [41]:
# for timing info 
total_starttime = time.time() 

# loop 
for sensorID in selectd_sensor_ids:
    print("--- working on sensor id "+str( sensorID )+" - at time "+str( time.ctime() )+" -- "+str( time.time() - total_starttime )+" s since start " )
    
    # partial timing 
    partial_starttime = time.time() 
    
    # get all the rows for this sensorID 
    df_w_all_values_of_curr_sensor_id = df_sensorData[ df_sensorData.sensor_id == sensorID ]
    ## print("\t - got all rows for curr sensor id - shape : "+str( df_w_all_values_of_curr_sensor_id.shape ) )
    #+" - at time "+( time.time() - partial_starttime)+" s" )
    
    # make sure the timestamp is there? 
    ## df_w_all_values_of_curr_sensor_id = df_w_all_values_of_curr_sensor_id.set_index("timestamp")
    
    # reduce columns for simplicity 
    df_w_all_values_of_curr_sensor_id = df_w_all_values_of_curr_sensor_id[ ['p1', 'p2'] ]
    ##print("\t - reduced column count. things now look like this : ")
    ##print( df_w_all_values_of_curr_sensor_id )
    
    
    # add start date to data time series
    df_w_all_values_of_curr_sensor_id = in_data__START_TIME__blank_row.append( df_w_all_values_of_curr_sensor_id )
    
    # add end date to data time series
    df_w_all_values_of_curr_sensor_id = df_w_all_values_of_curr_sensor_id.append( in_data__END_TIME__blank_row )
    
    ##print("\t - added start and end dates - df looks like this now : ")
    ## print( df_w_all_values_of_curr_sensor_id )
    
    # resample to the appropriate time length 
    df_w_all_values_of_curr_sensor_id__mean = df_w_all_values_of_curr_sensor_id.resample( sensor_uptime_sampling_freq ).mean().fillna( 0 )
    df_w_all_values_of_curr_sensor_id__median = df_w_all_values_of_curr_sensor_id.resample( sensor_uptime_sampling_freq ).median().fillna( 0 )
    ## print("\t - resampled in "+str( ( time.time() - partial_starttime ) )+"s")
    
#     print( df_w_all_values_of_curr_sensor_id )

    # - add the resampled data to the out array with all the sensor data 
    df_sensorids_columns_w_sensor_readings_2pt5_mins__mean[ sensorID ] = df_w_all_values_of_curr_sensor_id__mean['p1']
    df_sensorids_columns_w_sensor_readings_2pt5_mins__median[ sensorID ] = df_w_all_values_of_curr_sensor_id__median['p1']
    
    
    
print("\n \\\\\\\\\ and that all took "+str( time.time() - total_starttime )+" s :-) ")
    
    
    
    
    

--- working on sensor id 14296 - at time Tue Jun  2 08:10:52 2020 -- 0.0003502368927001953 s since start 
--- working on sensor id 17650 - at time Tue Jun  2 08:10:52 2020 -- 0.05527520179748535 s since start 
--- working on sensor id 19597 - at time Tue Jun  2 08:10:52 2020 -- 0.12896418571472168 s since start 
--- working on sensor id 16147 - at time Tue Jun  2 08:10:52 2020 -- 0.1839733123779297 s since start 
--- working on sensor id 12673 - at time Tue Jun  2 08:10:52 2020 -- 0.2430586814880371 s since start 
--- working on sensor id 12137 - at time Tue Jun  2 08:10:52 2020 -- 0.2880539894104004 s since start 
--- working on sensor id 18820 - at time Tue Jun  2 08:10:52 2020 -- 0.33388543128967285 s since start 
--- working on sensor id 10723 - at time Tue Jun  2 08:10:52 2020 -- 0.3874988555908203 s since start 
--- working on sensor id 10823 - at time Tue Jun  2 08:10:52 2020 -- 0.4347958564758301 s since start 
--- working on sensor id 12129 - at time Tue Jun  2 08:10:52 2020 -

#### check the output of the 2.5 min sensor readings - MEAN version

In [42]:
df_sensorids_columns_w_sensor_readings_2pt5_mins__mean.head()

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,27.07,8.60,22.33,56.07,51.03,31.67,40.88,21.80,33.23,9.33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:02:30,29.57,19.30,16.10,61.43,51.17,29.93,38.60,20.13,27.80,5.33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:05:00,24.57,12.70,14.47,51.13,44.57,32.00,32.47,16.80,23.83,7.43,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:07:30,25.30,8.80,15.07,53.27,43.73,23.63,33.90,18.97,23.47,7.30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:10:00,25.03,6.43,15.67,31.47,42.17,22.50,29.97,16.40,27.07,6.47,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
df_sensorids_columns_w_sensor_readings_2pt5_mins__mean.tail()

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-12-31 23:50:00,0.0,1.73,0.0,10.07,0.0,0.0,4.63,2.67,0.0,2.20,...,0.0,0.0,0.0,0.0,3.60,11.63,0.0,0.0,1.27,6.53
2019-12-31 23:52:30,0.0,1.33,0.0,5.63,0.0,0.0,5.80,3.20,0.0,3.00,...,0.0,0.0,0.0,0.0,4.37,4.09,0.0,0.0,1.10,12.07
2019-12-31 23:55:00,0.0,1.43,0.0,4.73,0.0,0.0,2.83,1.77,0.0,3.53,...,0.0,0.0,0.0,0.0,3.63,1.35,0.0,0.0,1.33,4.95
2019-12-31 23:57:30,0.0,0.00,0.0,3.87,0.0,0.0,6.37,4.17,0.0,3.20,...,0.0,0.0,0.0,0.0,4.53,8.93,0.0,0.0,0.85,7.78
2020-01-01 00:00:00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00


#### check the output of the 2.5 min sensor readings - MEDIAN version

In [44]:
df_sensorids_columns_w_sensor_readings_2pt5_mins__median.head()

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,27.07,8.60,22.33,56.07,51.03,31.67,40.88,21.80,33.23,9.33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:02:30,29.57,19.30,16.10,61.43,51.17,29.93,38.60,20.13,27.80,5.33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:05:00,24.57,12.70,14.47,51.13,44.57,32.00,32.47,16.80,23.83,7.43,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:07:30,25.30,8.80,15.07,53.27,43.73,23.63,33.90,18.97,23.47,7.30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:10:00,25.03,6.43,15.67,31.47,42.17,22.50,29.97,16.40,27.07,6.47,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
df_sensorids_columns_w_sensor_readings_2pt5_mins__median.tail()

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-12-31 23:50:00,0.0,1.73,0.0,10.07,0.0,0.0,4.63,2.67,0.0,2.20,...,0.0,0.0,0.0,0.0,3.60,11.63,0.0,0.0,1.27,6.53
2019-12-31 23:52:30,0.0,1.33,0.0,5.63,0.0,0.0,5.80,3.20,0.0,3.00,...,0.0,0.0,0.0,0.0,4.37,4.09,0.0,0.0,1.10,12.07
2019-12-31 23:55:00,0.0,1.43,0.0,4.73,0.0,0.0,2.83,1.77,0.0,3.53,...,0.0,0.0,0.0,0.0,3.63,1.35,0.0,0.0,1.33,4.95
2019-12-31 23:57:30,0.0,0.00,0.0,3.87,0.0,0.0,6.37,4.17,0.0,3.20,...,0.0,0.0,0.0,0.0,4.53,8.93,0.0,0.0,0.85,7.78
2020-01-01 00:00:00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00


### Find periods with 75% uptime in an hour, 90% uptime hours in a year 

#### setup the uptime statistics df, in the same shape as the sensor readings df
- so one can make an binary index of uptime, copying the sensor readings structure


##### how? - to find which sensors have been reporting at least 75% of each hour, 90% of the year : 
- make a equivalent dimension table where each cell with a reading gets a 1 in the cell
- convert the time period from 2.5 mins to 60 mins, and add all the cell values in each cell, such that a cell will have 24 readings per hour in ideal circumstances
- get the uptime per hour by divining the cumulative hourly cell number ( above ) by the max number of readings per hour. eg 24/24 for full uptime.
- get yearly uptime by : 
- - convert each hourly cell that has uptime of at least 75% into a 1. 
- - count the number of 1's per year compared to total number of hours in year. 
- use the annual statistics to choose which sensors (columns) should be included in the final statistics. 

##### how? - to get mean and mean sensor count readings, annually? 
- include only relevant cells by using the annual hourly index grid ( the one with a 1 for each cell that has relevant hourly uptime ) to multiply the equivalent hourly table of sensor readings. 
- - Thus each cell with relevant uptime stays the same, and non-qualifying cells get multiplied by 0 and become 0. 
- - Use pandas .median and .mean functions to summarise the values for each relevant sensor ( ie column ). 

In [46]:
# copy the sensor readings table, 
# to later make an uptime statistics table of equivalent dimensions
%time df_sensorids_columns_w_sensor_uptime_info_2pt5_mins = df_sensorids_columns_w_sensor_readings_2pt5_mins__mean.copy()

CPU times: user 21.6 ms, sys: 27.6 ms, total: 49.2 ms
Wall time: 48.3 ms


In [47]:
# convert all the cells with a sensor data reading larger than 0, into 1 
# ( then we can count the time slots with a sensor value )
%time df_sensorids_columns_w_sensor_uptime_info_2pt5_mins[ df_sensorids_columns_w_sensor_uptime_info_2pt5_mins > 0 ] = 1

CPU times: user 217 ms, sys: 48 ms, total: 265 ms
Wall time: 264 ms


In [48]:
# check what things look like 
df_sensorids_columns_w_sensor_uptime_info_2pt5_mins.head()

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:02:30,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:05:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:07:30,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:10:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
# get statistics 
## df_sensorids_columns_w_sensor_uptime_info_2pt5_mins.describe()

In [50]:
# rough calculation : how many 2.5m periods, out of the whole year, had measurements? 

# NOTE 90% hourly coverage ( where each hour has >75% coverage - that comes later!)
# test : get percentage of year covered by sensor - just in terms of random readings - 
( df_sensorids_columns_w_sensor_uptime_info_2pt5_mins.sum() / df_sensorids_columns_w_sensor_uptime_info_2pt5_mins.shape[0] ).sort_values( ascending=False )

7597     0.957301
12129    0.944473
14133    0.934242
16147    0.927112
12141    0.925657
19788    0.917371
12393    0.916092
18820    0.901432
18828    0.893570
10823    0.886216
7426     0.852645
19778    0.839960
22082    0.810860
19597    0.793851
10727    0.793052
17650    0.779815
20259    0.775952
20265    0.733753
14837    0.707835
9436     0.702575
12137    0.685080
12683    0.628617
14300    0.625392
11372    0.617900
25657    0.615351
10723    0.594955
26131    0.586969
20255    0.570964
12395    0.551986
12673    0.505839
13485    0.469994
18832    0.427143
23858    0.400745
9344     0.396193
18830    0.328076
30940    0.310253
26423    0.306106
25862    0.288835
28028    0.276759
20257    0.275950
27221    0.262190
27734    0.163712
31848    0.085178
31846    0.072084
14296    0.060050
35747    0.054504
36089    0.034499
7695     0.001969
22074    0.000200
22076    0.000067
dtype: float64

### now do the hourly calculations 
- find hourly uptime
- select only hours with sufficient uptime per hour 
- get p10 data from these

#### change sampling rate for both uptime and sensors data 


In [51]:
# test / check 
df_sensorids_columns_w_sensor_readings_2pt5_mins__mean.head()

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,27.07,8.60,22.33,56.07,51.03,31.67,40.88,21.80,33.23,9.33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:02:30,29.57,19.30,16.10,61.43,51.17,29.93,38.60,20.13,27.80,5.33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:05:00,24.57,12.70,14.47,51.13,44.57,32.00,32.47,16.80,23.83,7.43,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:07:30,25.30,8.80,15.07,53.27,43.73,23.63,33.90,18.97,23.47,7.30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:10:00,25.03,6.43,15.67,31.47,42.17,22.50,29.97,16.40,27.07,6.47,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
# resample sensor values into 60 min periods - sum the results - MEAN version
%time df_sensorids_columns_w_sensor_readings_60_mins__mean = df_sensorids_columns_w_sensor_readings_2pt5_mins__mean.resample( output_table_final_freq ).mean()
df_sensorids_columns_w_sensor_readings_60_mins__mean.head()


CPU times: user 80.9 ms, sys: 3.13 ms, total: 84 ms
Wall time: 82.8 ms


,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,18.392917,9.362917,13.369167,27.185000,29.342917,18.467917,22.897083,14.691250,16.983750,5.294167,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 01:00:00,10.402500,7.220000,10.125208,9.344583,9.372500,9.262708,19.510417,11.597708,9.155833,4.404375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 02:00:00,8.292500,5.723750,9.079583,4.192083,7.779583,8.031250,10.096458,11.105833,8.049583,3.870417,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 03:00:00,10.537083,6.585833,9.809583,10.816667,8.702083,7.798750,13.593750,4.109583,8.524583,3.947917,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 04:00:00,10.958333,7.516667,9.865417,12.065833,8.894583,8.485208,12.706667,12.502917,10.402083,3.967500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
# TEST
# TEST - trying to calculate mean values 'manually'
# TEST

# resample sensor values into 60 min periods - sum the results - MEAN version
%time df_sensorids_columns_w_sensor_readings_60_mins__mean__TRT2 = df_sensorids_columns_w_sensor_readings_2pt5_mins__mean.resample( output_table_final_freq ).sum()
df_sensorids_columns_w_sensor_readings_60_mins__mean.head()


CPU times: user 67.6 ms, sys: 3.5 ms, total: 71.1 ms
Wall time: 69.6 ms


,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,18.392917,9.362917,13.369167,27.185000,29.342917,18.467917,22.897083,14.691250,16.983750,5.294167,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 01:00:00,10.402500,7.220000,10.125208,9.344583,9.372500,9.262708,19.510417,11.597708,9.155833,4.404375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 02:00:00,8.292500,5.723750,9.079583,4.192083,7.779583,8.031250,10.096458,11.105833,8.049583,3.870417,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 03:00:00,10.537083,6.585833,9.809583,10.816667,8.702083,7.798750,13.593750,4.109583,8.524583,3.947917,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 04:00:00,10.958333,7.516667,9.865417,12.065833,8.894583,8.485208,12.706667,12.502917,10.402083,3.967500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
# resample sensor values into 60 min periods - sum the results - MEDIAN version
%time df_sensorids_columns_w_sensor_readings_60_mins__median = df_sensorids_columns_w_sensor_readings_2pt5_mins__mean.resample( output_table_final_freq ).median()
df_sensorids_columns_w_sensor_readings_60_mins__median.head()


CPU times: user 151 ms, sys: 23.6 ms, total: 174 ms
Wall time: 172 ms


,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,15.830,9.035,14.030,30.435,26.935,18.0000,20.115,14.400,14.215,5.100,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 01:00:00,9.835,6.615,10.265,10.520,9.450,8.7925,12.825,11.100,9.230,4.235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 02:00:00,7.765,5.850,8.765,2.265,7.820,8.0700,9.740,10.665,8.470,3.750,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 03:00:00,9.515,6.170,9.765,11.250,8.150,7.6500,11.600,0.000,8.685,3.720,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 04:00:00,11.130,6.485,10.270,12.835,8.500,8.5150,12.235,12.370,10.770,3.835,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
# uptime 
# -- count/sum the periods with a sensor measurement, in an hour, into one number
# ( then we can make a calculation uptime per hour )
%time df_sensorids_columns_w_sensor_uptime_info_60_mins = df_sensorids_columns_w_sensor_uptime_info_2pt5_mins.resample( output_table_final_freq ).sum().fillna(0)
df_sensorids_columns_w_sensor_uptime_info_60_mins.head()

CPU times: user 84.1 ms, sys: 0 ns, total: 84.1 ms
Wall time: 82.1 ms


,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,24.0,24.0,24.0,19.0,24.0,24.0,24.0,24.0,24.0,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 01:00:00,24.0,23.0,24.0,20.0,24.0,24.0,24.0,24.0,24.0,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 02:00:00,24.0,24.0,24.0,12.0,24.0,24.0,24.0,24.0,24.0,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 03:00:00,24.0,24.0,24.0,21.0,24.0,24.0,24.0,9.0,24.0,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 04:00:00,23.0,22.0,24.0,23.0,24.0,23.0,24.0,24.0,24.0,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
# get hourly uptime as a decimal 
# ( count of measurements per hour, divided by possible measurements per hour ( ca 24, for 2.5 min sampling rates ))
# -- ie the amount of readings, per hour, of a sensor
%time df_sensorids_columns_w_sensor_uptime_info_60_mins__as_decimal_val = df_sensorids_columns_w_sensor_uptime_info_60_mins.copy() / sensor_periods_in_hour
df_sensorids_columns_w_sensor_uptime_info_60_mins__as_decimal_val.head()

CPU times: user 51.4 ms, sys: 34.1 ms, total: 85.5 ms
Wall time: 14.8 ms


,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,1.000000,1.000000,1.0,0.791667,1.0,1.000000,1.0,1.000,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 01:00:00,1.000000,0.958333,1.0,0.833333,1.0,1.000000,1.0,1.000,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 02:00:00,1.000000,1.000000,1.0,0.500000,1.0,1.000000,1.0,1.000,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 03:00:00,1.000000,1.000000,1.0,0.875000,1.0,1.000000,1.0,0.375,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 04:00:00,0.958333,0.916667,1.0,0.958333,1.0,0.958333,1.0,1.000,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### get uptime above a certain decimal/percent ( 75% ) per hour, as binary 

In [57]:
# make a copy first 
df_uptime_above_75pct_60min_intervals = df_sensorids_columns_w_sensor_uptime_info_60_mins__as_decimal_val.copy()
df_uptime_above_75pct_60min_intervals.head()

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,1.000000,1.000000,1.0,0.791667,1.0,1.000000,1.0,1.000,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 01:00:00,1.000000,0.958333,1.0,0.833333,1.0,1.000000,1.0,1.000,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 02:00:00,1.000000,1.000000,1.0,0.500000,1.0,1.000000,1.0,1.000,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 03:00:00,1.000000,1.000000,1.0,0.875000,1.0,1.000000,1.0,0.375,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 04:00:00,0.958333,0.916667,1.0,0.958333,1.0,0.958333,1.0,1.000,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
# then make the binary index of values above 75% coverage ( >= 0.75 )
# -- IE put a 1 where there's an hourly uptime of at least 75% 
%time df_uptime_above_75pct_60min_intervals[ df_sensorids_columns_w_sensor_uptime_info_60_mins__as_decimal_val >= 0.75 ] = 1
%time df_uptime_above_75pct_60min_intervals[ df_sensorids_columns_w_sensor_uptime_info_60_mins__as_decimal_val < 0.75 ] = 0
df_uptime_above_75pct_60min_intervals.head()

CPU times: user 10 ms, sys: 450 µs, total: 10.5 ms
Wall time: 9.92 ms
CPU times: user 9.92 ms, sys: 0 ns, total: 9.92 ms
Wall time: 9.52 ms


,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 01:00:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 02:00:00,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 03:00:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 04:00:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
# calculate amount of hours, per year, fulfilling at least 75% uptime per hour

uptime_hours_w_75pct_or_more_sensor_coverage = ( df_uptime_above_75pct_60min_intervals.sum() / df_uptime_above_75pct_60min_intervals.shape[0] ).sort_values( ascending=False )
uptime_hours_w_75pct_or_more_sensor_coverage

7597     0.958338
12129    0.945554
14133    0.936879
12141    0.935053
16147    0.928661
19788    0.919073
12393    0.916220
17650    0.907659
18820    0.902180
18828    0.900354
10823    0.885744
7426     0.847049
19778    0.829357
22082    0.814747
10727    0.792261
19597    0.789408
20259    0.776852
20265    0.733706
14837    0.703687
12137    0.689990
9436     0.663737
11372    0.623102
25657    0.618651
14300    0.601530
26131    0.594909
10723    0.574706
20255    0.552334
12395    0.540121
12683    0.520945
12673    0.481908
13485    0.473919
18832    0.426778
9344     0.386486
23858    0.357037
30940    0.311380
18830    0.302477
26423    0.298139
25862    0.293574
28028    0.277822
20257    0.265951
27221    0.259902
27734    0.151695
31848    0.075790
31846    0.071111
14296    0.060952
35747    0.053647
36089    0.034357
7695     0.001826
22074    0.000000
22076    0.000000
dtype: float64

In [60]:
uptime_hours_w_75pct_or_more_sensor_coverage.index

Int64Index([ 7597, 12129, 14133, 12141, 16147, 19788, 12393, 17650, 18820,
            18828, 10823,  7426, 19778, 22082, 10727, 19597, 20259, 20265,
            14837, 12137,  9436, 11372, 25657, 14300, 26131, 10723, 20255,
            12395, 12683, 12673, 13485, 18832,  9344, 23858, 30940, 18830,
            26423, 25862, 28028, 20257, 27221, 27734, 31848, 31846, 14296,
            35747, 36089,  7695, 22074, 22076],
           dtype='int64')

In [61]:
# test - array access
uptime_hours_w_75pct_or_more_sensor_coverage[ 7597 ]

0.9583380892592169

In [62]:
# make a list of sensors with >= 90% coverage 
sensor_id_list_w_more_than_90pct_coverage = []

for sensor_id in uptime_hours_w_75pct_or_more_sensor_coverage.index:
    if uptime_hours_w_75pct_or_more_sensor_coverage[ sensor_id ] >= 0.9 : 
        sensor_id_list_w_more_than_90pct_coverage.append( sensor_id )
        
sensor_id_list_w_more_than_90pct_coverage

[7597, 12129, 14133, 12141, 16147, 19788, 12393, 17650, 18820, 18828]

In [63]:
# list of sensors with at least 90% sensor covarage
# ( where at least 75% of each hour has readings )
sensor_id_list_w_more_than_90pct_coverage

[7597, 12129, 14133, 12141, 16147, 19788, 12393, 17650, 18820, 18828]

### assemble relevant sensor (and uptime) data means and medians 

In [64]:
# only include columns of sensors with 75%hourly at 90% a year, coverage - uptime table
df_sensorids_columns_w_sensor_uptime_info_60_mins__w_only_relev_sensors = df_sensorids_columns_w_sensor_uptime_info_60_mins[ sensor_id_list_w_more_than_90pct_coverage ]
df_sensorids_columns_w_sensor_uptime_info_60_mins__w_only_relev_sensors.head()


,7597,12129,14133,12141,16147,19788,12393,17650,18820,18828
2019-01-01 00:00:00,24.0,24.0,21.0,24.0,19.0,0.0,24.0,24.0,24.0,0.0
2019-01-01 01:00:00,24.0,24.0,21.0,24.0,20.0,0.0,24.0,23.0,24.0,0.0
2019-01-01 02:00:00,24.0,24.0,21.0,24.0,12.0,0.0,24.0,24.0,24.0,0.0
2019-01-01 03:00:00,24.0,24.0,23.0,23.0,21.0,0.0,24.0,24.0,24.0,0.0
2019-01-01 04:00:00,24.0,24.0,22.0,24.0,23.0,0.0,24.0,22.0,24.0,0.0


In [65]:
# extract only relevant sensors for binary table of hourly readings >= 75% - make binary cell index 
%time df_uptime_above_75pct_60min_intervals__w_only_relev_sensors = df_uptime_above_75pct_60min_intervals[ sensor_id_list_w_more_than_90pct_coverage ]
df_uptime_above_75pct_60min_intervals__w_only_relev_sensors.head()

CPU times: user 1.62 ms, sys: 372 µs, total: 1.99 ms
Wall time: 1.84 ms


,7597,12129,14133,12141,16147,19788,12393,17650,18820,18828
2019-01-01 00:00:00,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
2019-01-01 01:00:00,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
2019-01-01 02:00:00,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
2019-01-01 03:00:00,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
2019-01-01 04:00:00,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0


In [66]:
# only include columns with sensors with enough sensor uptime -data
# - use above table as a mask for copying only relavant data from sensors data table to new table.
%time df_sensor_data_w_only_relev_sensors__mean = %time df_sensorids_columns_w_sensor_readings_60_mins__mean[ sensor_id_list_w_more_than_90pct_coverage ]
df_sensor_data_w_only_relev_sensors__mean.head()



CPU times: user 1.3 ms, sys: 298 µs, total: 1.6 ms
Wall time: 1.52 ms
CPU times: user 2.84 ms, sys: 362 µs, total: 3.2 ms
Wall time: 2.79 ms


,7597,12129,14133,12141,16147,19788,12393,17650,18820,18828
2019-01-01 00:00:00,19.667500,5.294167,29.777500,9.364583,27.185000,0.0,17.869167,9.362917,22.897083,0.0
2019-01-01 01:00:00,11.839167,4.404375,11.291250,7.302083,9.344583,0.0,7.527500,7.220000,19.510417,0.0
2019-01-01 02:00:00,8.600625,3.870417,8.766250,6.487083,4.192083,0.0,6.561667,5.723750,10.096458,0.0
2019-01-01 03:00:00,9.789167,3.947917,12.055833,6.877917,10.816667,0.0,7.090833,6.585833,13.593750,0.0
2019-01-01 04:00:00,11.420417,3.967500,13.006667,7.572500,12.065833,0.0,8.682083,7.516667,12.706667,0.0


### set cell with insufficient coverage ( less than 75% per hour ) to zero - mean version

## *** Annual hourly mean readings 

In [67]:
df_sensor_data_w_only_relev_sensors__with_relev_hourly_coverage__mean = df_sensor_data_w_only_relev_sensors__mean * df_uptime_above_75pct_60min_intervals__w_only_relev_sensors
df_sensor_data_w_only_relev_sensors__with_relev_hourly_coverage__mean.head()

,7597,12129,14133,12141,16147,19788,12393,17650,18820,18828
2019-01-01 00:00:00,19.667500,5.294167,29.777500,9.364583,27.185000,0.0,17.869167,9.362917,22.897083,0.0
2019-01-01 01:00:00,11.839167,4.404375,11.291250,7.302083,9.344583,0.0,7.527500,7.220000,19.510417,0.0
2019-01-01 02:00:00,8.600625,3.870417,8.766250,6.487083,0.000000,0.0,6.561667,5.723750,10.096458,0.0
2019-01-01 03:00:00,9.789167,3.947917,12.055833,6.877917,10.816667,0.0,7.090833,6.585833,13.593750,0.0
2019-01-01 04:00:00,11.420417,3.967500,13.006667,7.572500,12.065833,0.0,8.682083,7.516667,12.706667,0.0


In [68]:
# save
%time df_sensor_data_w_only_relev_sensors__with_relev_hourly_coverage__mean.to_csv("2019_stockholm_annual_hourly_sensor_values__MEAN.csv")

CPU times: user 247 ms, sys: 7.5 ms, total: 255 ms
Wall time: 369 ms


## *** Annual hourly median readings 

In [69]:
### set all non >= 75% coverage cells to 0 / zero
df_sensorids_columns_w_sensor_readings_60_mins__median__w_only_cells_w_relev_coverage = df_sensorids_columns_w_sensor_readings_60_mins__median * df_uptime_above_75pct_60min_intervals__w_only_relev_sensors;
df_sensorids_columns_w_sensor_readings_60_mins__median__w_only_cells_w_relev_coverage.head()

,7426,7597,7695,9344,9436,10723,10727,10823,11372,12129,...,26131,26423,27221,27734,28028,30940,31846,31848,35747,36089
2019-01-01 00:00:00,NaN,16.5200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01 01:00:00,NaN,11.4500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.235,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01 02:00:00,NaN,8.0075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01 03:00:00,NaN,9.0150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.720,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01 04:00:00,NaN,11.6150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.835,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
df_sensorids_columns_w_sensor_readings_60_mins__median__w_only_cells_w_relev_coverage.to_csv("2019_stockholm_annual_hourly_sensor_values__MEDIAN.csv")

## *** Per sensor annual median value

In [71]:
# calculating median for whole year, using reshape
df_sensorids_columns_w_sensor_readings_60_mins__median__w_only_cells_w_relev_coverage.resample( "366D" ).median()

,7426,7597,7695,9344,9436,10723,10727,10823,11372,12129,...,26131,26423,27221,27734,28028,30940,31846,31848,35747,36089
2019-01-01,NaN,4.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.565,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
# doing the same as above, but in a different way
# - SPOILER : one gets the same results 
df_1_year_median_value_per_column = df_sensorids_columns_w_sensor_readings_60_mins__median__w_only_cells_w_relev_coverage.median()
df_1_year_median_value_per_column

7426       NaN
7597     4.020
7695       NaN
9344       NaN
9436       NaN
10723      NaN
10727      NaN
10823      NaN
11372      NaN
12129    3.565
12137      NaN
12141    3.600
12393    3.635
12395      NaN
12673      NaN
12683      NaN
13485      NaN
14133    6.100
14296      NaN
14300      NaN
14837      NaN
16147    5.290
17650    2.550
18820    5.200
18828    3.700
18830      NaN
18832      NaN
19597      NaN
19778      NaN
19788    2.850
20255      NaN
20257      NaN
20259      NaN
20265      NaN
22074      NaN
22076      NaN
22082      NaN
23858      NaN
25657      NaN
25862      NaN
26131      NaN
26423      NaN
27221      NaN
27734      NaN
28028      NaN
30940      NaN
31846      NaN
31848      NaN
35747      NaN
36089      NaN
dtype: float64

## ** save : Annual median value per sensor 

In [73]:
# save annual median value ( one value per sensor )
df_1_year_median_value_per_column.to_csv("2019_stockholm_annual_sensor_value__one_numer_per_sensor__MEDIAN.csv")

In [74]:
# TEST WITH REPLACE 0 with np.nan - to see if it affect median
%time df_60min_median_sensor_data__with_nan = df_sensorids_columns_w_sensor_readings_60_mins__median__w_only_cells_w_relev_coverage.replace(0,np.nan )
df_60min_median_sensor_data__with_nan__1_YEAR_VALUE = df_sensorids_columns_w_sensor_readings_60_mins__median__w_only_cells_w_relev_coverage.resample( "366D" ).median()
df_60min_median_sensor_data__with_nan__1_YEAR_VALUE

CPU times: user 1.9 ms, sys: 417 µs, total: 2.32 ms
Wall time: 1.87 ms


,7426,7597,7695,9344,9436,10723,10727,10823,11372,12129,...,26131,26423,27221,27734,28028,30940,31846,31848,35747,36089
2019-01-01,NaN,4.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.565,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### TEST : 'manual' mean calculation

In [75]:
df_sensorids_columns_w_sensor_readings_60_mins__mean__TRT2_only_relev_sensors = df_sensorids_columns_w_sensor_readings_60_mins__mean__TRT2[ sensor_id_list_w_more_than_90pct_coverage ]

In [76]:
df_sensorids_columns_w_sensor_readings_60_mins__mean__TRT2_only_relev_sensors__MEAN = df_sensorids_columns_w_sensor_readings_60_mins__mean__TRT2_only_relev_sensors / df_sensorids_columns_w_sensor_uptime_info_60_mins__w_only_relev_sensors
df_sensorids_columns_w_sensor_readings_60_mins__mean__TRT2_only_relev_sensors__MEAN.head()

,7597,12129,14133,12141,16147,19788,12393,17650,18820,18828
2019-01-01 00:00:00,19.667500,5.294167,34.031429,9.364583,34.338947,NaN,17.869167,9.362917,22.897083,NaN
2019-01-01 01:00:00,11.839167,4.404375,12.904286,7.302083,11.213500,NaN,7.527500,7.533913,19.510417,NaN
2019-01-01 02:00:00,8.600625,3.870417,10.018571,6.487083,8.384167,NaN,6.561667,5.723750,10.096458,NaN
2019-01-01 03:00:00,9.789167,3.947917,12.580000,7.176957,12.361905,NaN,7.090833,6.585833,13.593750,NaN
2019-01-01 04:00:00,11.420417,3.967500,14.189091,7.572500,12.590435,NaN,8.682083,8.200000,12.706667,NaN


In [77]:
# remove cells with insufficient readings… per hour
df_sensorids_columns_w_sensor_readings_60_mins__mean__TRT2_only_relev_sensors__MEAN =  df_sensorids_columns_w_sensor_readings_60_mins__mean__TRT2_only_relev_sensors__MEAN * df_uptime_above_75pct_60min_intervals__w_only_relev_sensors;

In [78]:
# the version with .mean done in the resampling from 2.5m -> 60m 
df_sensorids_columns_w_sensor_readings_60_mins__mean__TRT2_only_relev_sensors__MEAN.head()

,7597,12129,14133,12141,16147,19788,12393,17650,18820,18828
2019-01-01 00:00:00,19.667500,5.294167,34.031429,9.364583,34.338947,NaN,17.869167,9.362917,22.897083,NaN
2019-01-01 01:00:00,11.839167,4.404375,12.904286,7.302083,11.213500,NaN,7.527500,7.533913,19.510417,NaN
2019-01-01 02:00:00,8.600625,3.870417,10.018571,6.487083,0.000000,NaN,6.561667,5.723750,10.096458,NaN
2019-01-01 03:00:00,9.789167,3.947917,12.580000,7.176957,12.361905,NaN,7.090833,6.585833,13.593750,NaN
2019-01-01 04:00:00,11.420417,3.967500,14.189091,7.572500,12.590435,NaN,8.682083,8.200000,12.706667,NaN


### and now… find the mean/annual averages for each sensor … 
#### - remembering to divide the sum of readings by number of valid sensor readings 

In [79]:
# get the sum of sensor values, per year - MEAN
%time df_sum_of_valid_hourly_mean_sensor_values_per_year = df_sensorids_columns_w_sensor_readings_60_mins__mean__TRT2_only_relev_sensors__MEAN.sum()
df_sum_of_valid_hourly_mean_sensor_values_per_year

CPU times: user 1.69 ms, sys: 369 µs, total: 2.05 ms
Wall time: 1.64 ms


7597     67919.999327
12129    51293.818788
14133    98101.460159
12141    48196.160688
16147    83819.187854
19788    48305.342772
12393    50199.267064
17650    48952.837113
18820    81821.580742
18828    65949.806141
dtype: float64

## *** Annual mean value, per sensor

In [80]:
# get the number of readings per year 
df_uptime_count_of_valid_readings_per_year = df_uptime_above_75pct_60min_intervals__w_only_relev_sensors.sum()
df_uptime_count_of_valid_readings_per_year

7597     8396.0
12129    8284.0
14133    8208.0
12141    8192.0
16147    8136.0
19788    8052.0
12393    8027.0
17650    7952.0
18820    7904.0
18828    7888.0
dtype: float64

In [81]:
# - get the MEAN /average
mean_sensor_value_per_sensor_and_year = df_sum_of_valid_hourly_mean_sensor_values_per_year / df_uptime_count_of_valid_readings_per_year
mean_sensor_value_per_sensor_and_year

7597      8.089566
12129     6.191914
14133    11.951932
12141     5.883320
16147    10.302260
19788     5.999173
12393     6.253802
17650     6.156041
18820    10.351921
18828     8.360777
dtype: float64

## *** save : annual mean value, per sensor 

In [82]:
mean_sensor_value_per_sensor_and_year.to_csv( "2019_stockholm_annual_sensor_value__one_numer_per_sensor__MEAN.csv" )

### Fetch lat-long locations of selected sensor

In [83]:
# reload data 
%time df_sensorData = pd.read_csv( curr_in_data_url )
# reduce columns for speed
df_sensorData = df_sensorData[ ['sensor_id', 'lat', 'lon'] ]

CPU times: user 6.95 s, sys: 1.07 s, total: 8.03 s
Wall time: 8.03 s


In [84]:
# make table to export data to 
lat_lon_of_sensors_w_sufficient_uptime = pd.DataFrame( index=sensor_id_list_w_more_than_90pct_coverage )
# add lat lon columns
lat_lon_of_sensors_w_sufficient_uptime['lat'] = np.nan
lat_lon_of_sensors_w_sufficient_uptime['lon'] = np.nan
lat_lon_of_sensors_w_sufficient_uptime

,lat,lon
7597,NaN,NaN
12129,NaN,NaN
14133,NaN,NaN
12141,NaN,NaN
16147,NaN,NaN
19788,NaN,NaN
12393,NaN,NaN
17650,NaN,NaN
18820,NaN,NaN
18828,NaN,NaN


In [85]:
# loop and find the lat lons of the sensors 
for sensor_id in sensor_id_list_w_more_than_90pct_coverage:
    # fetch the first row with this sensor int the data
    first_row_of_sensor = df_sensorData[ df_sensorData.sensor_id == sensor_id ].iloc[0]
    # set the data 
    lat_lon_of_sensors_w_sufficient_uptime.loc[ sensor_id, 'lat' ] = first_row_of_sensor.lat
    lat_lon_of_sensors_w_sufficient_uptime.loc[ sensor_id, 'lon' ] = first_row_of_sensor.lon

    

## *** Sensor coordinates

In [86]:
lat_lon_of_sensors_w_sufficient_uptime

,lat,lon
7597,59.321,18.064
12129,59.341,18.040
14133,59.364,18.018
12141,59.285,18.023
16147,59.364,18.018
19788,59.332,18.243
12393,59.343,18.033
17650,59.304,18.101
18820,59.333,18.033
18828,59.302,18.099


## ** save : sensor coordinates

In [87]:
lat_lon_of_sensors_w_sufficient_uptime.to_csv("2019_stockholm__sensor_coordinates.csv")

## Get percentile values 

### Criteria : 
##### - get daily median value, based on the hours with >= 75% coverage.
##### - get annual 90th percentile.

### how?

##### - get daily median value, based on the hours with >= 75% coverage.
- ¿ make calculations on hourly median values or median values from the qualifying 2.5 min values in the day?
- - make a bit index of days with 75% coverage of 75% of the hours. 
- - - ( count the hours with sufficient coverage, divide by 24/hours-in-the-day ). 
- - use qualifying day index to make a 2.5 min index of qualifying day. 
- - - ie. resample daily index to 180 seconds, and fill each qualifying day with 1s ( make sure the mask works btw ! ) 
- - - make more detailed index of qualifying hours ( use daily index mixed with valid values ). 
- - - use above index to remove hours with 
# 
#### - get 90th percentile : from the median values of each day 
#### - - 

#### NOTES : 

- appears pandas ignored nan values for quantile calculations
https://stackoverflow.com/questions/52171333/does-the-quantile-function-in-python-ignore-nan
and 
https://numpy.org/doc/stable/reference/generated/numpy.nanquantile.html

#### generate daily uptime indicies

- generate hourly uptime index ( where a 1 indicates enough uptime per hour )
- resample to daily intervals .sum() to get cumulative values per day
- divide by 24 ( hours in the day ) to get index per day
- make daily index based on these values

In [88]:
# could be a good table to start with 
df_uptime_above_75pct_60min_intervals.head()

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 01:00:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 02:00:00,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 03:00:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 04:00:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
# resample above count of measurements per qualifying 1 hour interval, 
# into a count of qualifying hours  per day
df_uptime_daily_cumulative_uptime_count_of_at_least_75pc_per_hour_coverage = df_uptime_above_75pct_60min_intervals.resample( "1D" ).sum()
df_uptime_daily_cumulative_uptime_count_of_at_least_75pc_per_hour_coverage.head()

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01,22.0,24.0,23.0,13.0,24.0,24.0,24.0,23.0,24.0,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-02,7.0,19.0,15.0,16.0,16.0,20.0,18.0,21.0,18.0,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-03,5.0,24.0,15.0,24.0,24.0,24.0,24.0,21.0,24.0,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-04,21.0,23.0,7.0,22.0,23.0,23.0,23.0,22.0,23.0,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-05,12.0,9.0,9.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
# get a decimal uptime per day 
# - divide number of qualifying meaurements, by 24 hours
df_uptime_daily_cumulative_uptime_count_of_at_least_75pc_per_hour_coverage__as_decimal = df_uptime_daily_cumulative_uptime_count_of_at_least_75pc_per_hour_coverage/24
df_uptime_daily_cumulative_uptime_count_of_at_least_75pc_per_hour_coverage__as_decimal

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01,0.916667,1.000000,0.958333,0.541667,1.000000,1.000000,1.000000,0.958333,1.000000,1.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2019-01-02,0.291667,0.791667,0.625000,0.666667,0.666667,0.833333,0.750000,0.875000,0.750000,0.750000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2019-01-03,0.208333,1.000000,0.625000,1.000000,1.000000,1.000000,1.000000,0.875000,1.000000,1.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2019-01-04,0.875000,0.958333,0.291667,0.916667,0.958333,0.958333,0.958333,0.916667,0.958333,0.958333,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2019-01-05,0.500000,0.375000,0.375000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-28,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,...,0.0,0.0,0.416667,0.0,1.0,1.0,0.0,0.0,1.000000,0.000000
2019-12-29,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,...,0.0,0.0,0.583333,0.0,1.0,1.0,0.0,0.0,1.000000,0.291667
2019-12-30,0.000000,0.958333,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,...,0.0,0.0,0.708333,0.0,1.0,1.0,0.0,0.0,1.000000,1.000000
2019-12-31,0.000000,0.875000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,...,0.0,0.0,0.000000,0.0,1.0,1.0,0.0,0.0,0.583333,1.000000


In [91]:
# make boolean index, per day, of days with at least 75% of the hours having at least 75% coverage

# start by copying the above table, to get the same format/structure 
df_uptime_daily_cumulative_uptime_count_of_at_least_75pc_per_hour_coverage__as_boolean = df_uptime_daily_cumulative_uptime_count_of_at_least_75pc_per_hour_coverage__as_decimal.copy()
# zero it - to have a clean slate
df_uptime_daily_cumulative_uptime_count_of_at_least_75pc_per_hour_coverage__as_boolean = df_uptime_daily_cumulative_uptime_count_of_at_least_75pc_per_hour_coverage__as_boolean * 0 
# show it 
df_uptime_daily_cumulative_uptime_count_of_at_least_75pc_per_hour_coverage__as_boolean

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-12-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
# set cells/time slots with qualifying uptime, to 1
df_uptime_daily_cumulative_uptime_count_of_at_least_75pc_per_hour_coverage__as_boolean[ df_uptime_daily_cumulative_uptime_count_of_at_least_75pc_per_hour_coverage__as_decimal >= 0.75 ]  = 1

In [93]:
# show
df_uptime_daily_cumulative_uptime_count_of_at_least_75pc_per_hour_coverage__as_boolean.head()

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-02,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-03,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-04,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [94]:
# show
df_uptime_daily_cumulative_uptime_count_of_at_least_75pc_per_hour_coverage__as_boolean.tail()

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-12-28,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
2019-12-29,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
2019-12-30,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
2019-12-31,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [95]:
# get valid-uptime index which can be applied onto hourly values, from the qualifying day index
# 
# resample the above index of qualifying days to 2.5 min sample periods,
# ( ie if a day has a 1 ( qualifies ), every hour then gets a 1 )
# so we could use it as an index for hours, whether the day is ok
# df_uptime_index_with_only_hours_for_days_w_sufficient_uptime =  df_uptime_daily_cumulative_uptime_count_of_at_least_75pc_per_hour_coverage__as_boolean.resample("3600S").pad().fillna(0)
df_uptime_index_with_only_sampling_periods_180s_for_days_w_sufficient_uptime =  df_uptime_daily_cumulative_uptime_count_of_at_least_75pc_per_hour_coverage__as_boolean.resample( sensor_uptime_sampling_freq ).pad().fillna(0)
df_uptime_index_with_only_sampling_periods_180s_for_days_w_sufficient_uptime

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:02:30,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:05:00,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:07:30,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:10:00,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 23:50:00,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
2019-12-31 23:52:30,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
2019-12-31 23:55:00,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
2019-12-31 23:57:30,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0


#### how to move on? 

- to only include relevant days : multiply the sensor readings by the (hourly) boolean
    index indicating whether a day has enough coverage
- to only include relevant hours ( with 75% coverage ) - multiply the hours by the hourly index of relevant uptime
( - replace 0 with nan? -- try both )

- to get daily medians : resample to day rates and ask for medians 

- to get annual percentiles : only select qualifying days and calcuclate the percentile based on those.


In [96]:
# set sampling periods in non-qualifying days to 0
# ( then they 'dont count' in our later calculations )
#
# multiply the sampling periods readings by the valid-day-hourly-boolean-index created above
# - to zero out the hours in irrelevant days 
df_sensor_values__zero_where_day_w_insufficient_uptime = df_sensorids_columns_w_sensor_readings_2pt5_mins__mean * df_uptime_index_with_only_sampling_periods_180s_for_days_w_sufficient_uptime
df_sensor_values__zero_where_day_w_insufficient_uptime.head()

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,27.07,8.60,22.33,0.0,51.03,31.67,40.88,21.80,33.23,9.33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:02:30,29.57,19.30,16.10,0.0,51.17,29.93,38.60,20.13,27.80,5.33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:05:00,24.57,12.70,14.47,0.0,44.57,32.00,32.47,16.80,23.83,7.43,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:07:30,25.30,8.80,15.07,0.0,43.73,23.63,33.90,18.97,23.47,7.30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:10:00,25.03,6.43,15.67,0.0,42.17,22.50,29.97,16.40,27.07,6.47,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
# - start work on only including qualifying sampling periods 
# ( ie sampling periods within hours with >= 75% coverage )
# - - here we resample the hourly index of qualifying hours, to 2.5 min sampling periods

# get index of which hours have enough uptime … 
# - multiply the sensor readings by the index of valid hours, to remove (zero) invalid hours 
df_uptime_above_75pct_2pt5min_intervals = df_uptime_above_75pct_60min_intervals.resample( sensor_uptime_sampling_freq ).pad()
# check the values - sensor 16147 should be zero for 02:00 - 03:00
df_uptime_above_75pct_2pt5min_intervals[ df_uptime_above_75pct_2pt5min_intervals.index > '2019-01-01 02:00' ].head()

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 02:02:30,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 02:05:00,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 02:07:30,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 02:10:00,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 02:12:30,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [98]:
# set non-qualifying sampling periods ( ie with less than 75% coverage per hour ) to 0 

# multiply sensor readings by the index of hours with relevant uptime
df_sensor_values__w_zeros_for_nonqualifying_hours_and_days = df_sensor_values__zero_where_day_w_insufficient_uptime * df_uptime_above_75pct_2pt5min_intervals
df_sensor_values__w_zeros_for_nonqualifying_hours_and_days[ df_sensor_values__w_zeros_for_nonqualifying_hours_and_days.index > '2019-01-02 23:00' ] 

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-02 23:02:30,0.0,1.80,0.0,0.00,0.0,2.47,3.80,3.90,2.97,3.30,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00
2019-01-02 23:05:00,0.0,2.13,0.0,0.00,0.0,2.87,3.03,2.90,3.13,8.77,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00
2019-01-02 23:07:30,0.0,0.00,0.0,0.00,0.0,1.70,7.57,3.10,6.27,0.00,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00
2019-01-02 23:10:00,0.0,1.60,0.0,0.00,0.0,5.00,3.43,5.97,4.80,4.30,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00
2019-01-02 23:12:30,0.0,1.20,0.0,0.00,0.0,4.50,4.27,4.20,6.00,3.23,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 23:50:00,0.0,1.73,0.0,10.07,0.0,0.00,4.63,2.67,0.00,2.20,...,0.0,0.0,0.0,0.0,3.60,11.63,0.0,0.0,0.0,6.53
2019-12-31 23:52:30,0.0,1.33,0.0,5.63,0.0,0.00,5.80,3.20,0.00,3.00,...,0.0,0.0,0.0,0.0,4.37,4.09,0.0,0.0,0.0,12.07
2019-12-31 23:55:00,0.0,1.43,0.0,4.73,0.0,0.00,2.83,1.77,0.00,3.53,...,0.0,0.0,0.0,0.0,3.63,1.35,0.0,0.0,0.0,4.95
2019-12-31 23:57:30,0.0,0.00,0.0,3.87,0.0,0.00,6.37,4.17,0.00,3.20,...,0.0,0.0,0.0,0.0,4.53,8.93,0.0,0.0,0.0,7.78


In [99]:
# - now get daily median values :) 
# - - resample 2.5 min periods to daily values, use median 
df_sensor_values__w_only_qualifying_uptime_hours_and_days__as_daily_median_value = df_sensor_values__w_zeros_for_nonqualifying_hours_and_days.resample( "1D" ).median()
df_sensor_values__w_only_qualifying_uptime_hours_and_days__as_daily_median_value.head()

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01,2.215,1.80,2.47,0.0,3.30,2.90,4.20,3.70,3.385,2.630,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-02,0.000,1.70,0.00,0.0,0.00,2.47,3.45,3.97,3.115,1.830,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-03,0.000,1.97,0.00,4.2,3.27,2.87,4.20,3.80,3.370,2.115,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-04,1.700,1.23,0.00,2.7,2.20,1.73,2.87,2.23,1.830,1.430,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-05,0.000,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [100]:
# try the above again, but with provisions for measurements with 0 
# ( ie try to make double-sure values of 0 are not included in median calculations )
# ( replace the 0's with nans )

# - replace zeros with nan
df_sensor_values__w_zeros_for_nonqualifying_hours_and_days__with_nan = df_sensor_values__w_zeros_for_nonqualifying_hours_and_days.replace( 0, np.nan )
df_sensor_values__w_zeros_for_nonqualifying_hours_and_days__with_nan.head()

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,27.07,8.60,22.33,NaN,51.03,31.67,40.88,21.80,33.23,9.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01 00:02:30,29.57,19.30,16.10,NaN,51.17,29.93,38.60,20.13,27.80,5.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01 00:05:00,24.57,12.70,14.47,NaN,44.57,32.00,32.47,16.80,23.83,7.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01 00:07:30,25.30,8.80,15.07,NaN,43.73,23.63,33.90,18.97,23.47,7.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01 00:10:00,25.03,6.43,15.67,NaN,42.17,22.50,29.97,16.40,27.07,6.47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
# now try getting the median, again 
# ( with data where 0s are replaced with nan's )
df_sensor_values__w_only_qualifying_uptime_hours_and_days__as_daily_median_value__with_nan__TRY2 = df_sensor_values__w_zeros_for_nonqualifying_hours_and_days__with_nan.resample( "1D" ).median()
df_sensor_values__w_only_qualifying_uptime_hours_and_days__as_daily_median_value__with_nan__TRY2.head()

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01,2.83,1.815,2.67,NaN,3.37,2.90,4.225,3.90,3.430,2.63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-02,NaN,2.200,NaN,NaN,NaN,2.77,4.330,4.33,3.800,2.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-03,NaN,2.050,NaN,4.3,3.30,2.90,4.415,4.13,3.415,2.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-04,2.03,1.300,NaN,3.0,2.33,1.87,2.970,2.37,1.930,1.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## ** save daily median values 


In [102]:
df_sensor_values__w_only_qualifying_uptime_hours_and_days__as_daily_median_value__with_nan__TRY2.to_csv("2019_stockholm_daily_median_values_where_daily_uptime_is_at_least_75pc_by_75pc.csv")

### Get annual 90th percentile values 


In [103]:
# based on daily median values 
df_annual_90th_percentile_values__based_on_daily_median_values = df_sensor_values__w_only_qualifying_uptime_hours_and_days__as_daily_median_value__with_nan__TRY2.quantile( 0.9, axis=0 )

In [104]:
df_annual_90th_percentile_values__based_on_daily_median_values

14296    16.967000
17650    10.695000
19597    14.377000
16147    17.930500
12673    17.194000
12137    10.927000
18820    16.029000
10723    15.998500
10823    12.620000
12129    11.064000
14837    13.270000
14133    19.421000
7597     14.409500
9436     10.404000
12393    10.930000
12141     9.885000
14300     7.938000
9344      8.644000
18830    18.170000
7426     14.530000
12395    10.940000
18832    17.866000
18828    15.004000
10727    11.370000
12683    13.650000
19778    16.526000
19788     9.835000
7695           NaN
20259    12.108000
20265    13.650000
20255    11.793000
20257    28.027000
22082    16.421000
22074          NaN
11372    10.685500
22076          NaN
23858     9.145000
25657    17.219500
25862    10.286000
26131    10.886000
26423    11.190000
27221    11.929000
27734     7.375000
28028     8.514583
13485    14.274000
30940    20.600000
31846     0.200000
31848     8.705500
35747     6.192500
36089    19.288500
Name: 0.9, dtype: float64

### now get the yearly statistics, per sensor 
#### - how many of the years' hours have >= 75 hours with at last 75% coverage per hour


In [105]:
df_annual_summary_per_sensor_of_days_with_at_least_75pc_hourly_uptime = df_uptime_daily_cumulative_uptime_count_of_at_least_75pc_per_hour_coverage__as_boolean.sum()/df_uptime_daily_cumulative_uptime_count_of_at_least_75pc_per_hour_coverage__as_boolean.shape[0]
df_annual_summary_per_sensor_of_days_with_at_least_75pc_hourly_uptime.sort_values( ascending=False )

7597     0.950820
12129    0.926230
14133    0.920765
12141    0.918033
16147    0.907104
12393    0.904372
19788    0.901639
17650    0.890710
18820    0.885246
10823    0.882514
18828    0.879781
7426     0.822404
22082    0.803279
19778    0.778689
19597    0.770492
20259    0.762295
10727    0.745902
20265    0.732240
14837    0.693989
12137    0.683060
9436     0.636612
25657    0.612022
11372    0.606557
26131    0.581967
10723    0.551913
14300    0.516393
12395    0.510929
20255    0.497268
12683    0.464481
13485    0.461749
12673    0.445355
18832    0.423497
9344     0.363388
30940    0.303279
25862    0.270492
28028    0.262295
23858    0.262295
27221    0.240437
26423    0.221311
18830    0.153005
27734    0.112022
20257    0.087432
31848    0.076503
31846    0.060109
14296    0.054645
35747    0.049180
36089    0.027322
7695     0.000000
22074    0.000000
22076    0.000000
dtype: float64

In [106]:
# get list of qualifying sensors 
df_sensor_list_w_at_least_75pc_hourly_coverage_at_least_75_of_year = df_annual_summary_per_sensor_of_days_with_at_least_75pc_hourly_uptime[ df_annual_summary_per_sensor_of_days_with_at_least_75pc_hourly_uptime >= 0.75 ].sort_values( ascending=False ) 
df_sensor_list_w_at_least_75pc_hourly_coverage_at_least_75_of_year

7597     0.950820
12129    0.926230
14133    0.920765
12141    0.918033
16147    0.907104
12393    0.904372
19788    0.901639
17650    0.890710
18820    0.885246
10823    0.882514
18828    0.879781
7426     0.822404
22082    0.803279
19778    0.778689
19597    0.770492
20259    0.762295
dtype: float64

### reduce sensor reading/statistics arrays to only sensors with relevant uptime

## *** 90th percentile values
### for all sensors with >= 75% daily uptime ( with 75% coverage per hour  )

In [107]:
df_annual_90th_percentile_values__based_on_daily_median_values__only_sensors_w_relevant_uptime = df_annual_90th_percentile_values__based_on_daily_median_values[ df_sensor_list_w_at_least_75pc_hourly_coverage_at_least_75_of_year.index ].sort_values( ascending=False )

In [108]:
df_annual_90th_percentile_values__based_on_daily_median_values__only_sensors_w_relevant_uptime

14133    19.4210
16147    17.9305
19778    16.5260
22082    16.4210
18820    16.0290
18828    15.0040
7426     14.5300
7597     14.4095
19597    14.3770
10823    12.6200
20259    12.1080
12129    11.0640
12393    10.9300
17650    10.6950
12141     9.8850
19788     9.8350
Name: 0.9, dtype: float64

## ** Save : 90th percentile values of sensors with relevant uptime 

In [149]:
df_annual_90th_percentile_values__based_on_daily_median_values__only_sensors_w_relevant_uptime.to_csv("df_annual_90th_percentile_values__based_on_daily_median_values__only_sensors_w_relevant_uptime.csv")

### *** 90th percentile values
#### for the 10 sensors with 90% uptime ( of hours with 75% coverage )

In [109]:
# and the same, but using the earlier list of sensors with 75% uptime all of the year. 
df_annual_90th_percentile_values__based_on_daily_median_values__w_previous_list_of_relevant_sensors = df_annual_90th_percentile_values__based_on_daily_median_values[ sensor_id_list_w_more_than_90pct_coverage ].sort_values( ascending=False )
df_annual_90th_percentile_values__based_on_daily_median_values__w_previous_list_of_relevant_sensors

14133    19.4210
16147    17.9305
18820    16.0290
18828    15.0040
7597     14.4095
12129    11.0640
12393    10.9300
17650    10.6950
12141     9.8850
19788     9.8350
Name: 0.9, dtype: float64

## ** Save : 90th percentile values
#### for the 10 sensors with 90% uptime ( of hours with 75% coverage )

In [110]:
df_annual_90th_percentile_values__based_on_daily_median_values__w_previous_list_of_relevant_sensors.to_csv("2019_stockholm__90th_percentile_values_for_10_sesnors_w_sufficient_uptime.csv")

### *** Daily median sensor values for hours with at least 75% uptime per day - only the 10 sensors with 90% uptime ( where 75% hourly uptime coverage qualifies ) 

In [111]:
## Daily median values 
#### based on hours with 75% coverage 
#### - only of the 10 sensors found to have sufficient coverage sensor_id_list_w_more_than_90pct_coverage

df_sensor_values__w_only_qualifying_uptime_hours_and_days__as_daily_median_value__with_nan__TRY2__only_10_sensors = df_sensor_values__w_only_qualifying_uptime_hours_and_days__as_daily_median_value__with_nan__TRY2[ sensor_id_list_w_more_than_90pct_coverage ]
df_sensor_values__w_only_qualifying_uptime_hours_and_days__as_daily_median_value__with_nan__TRY2__only_10_sensors.head()

,7597,12129,14133,12141,16147,19788,12393,17650,18820,18828
2019-01-01,2.385,2.63,4.37,2.540,NaN,NaN,3.12,1.815,4.225,NaN
2019-01-02,2.400,2.33,NaN,2.610,NaN,NaN,3.09,2.200,4.330,NaN
2019-01-03,2.100,2.20,4.20,2.800,4.3,NaN,2.90,2.050,4.415,NaN
2019-01-04,1.430,1.60,3.13,1.485,3.0,NaN,2.05,1.300,2.970,NaN
2019-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### ** Save : Daily median sensor values for hours with at least 75% uptime per day - only 10 sensors :-( 

In [112]:
df_sensor_values__w_only_qualifying_uptime_hours_and_days__as_daily_median_value__with_nan__TRY2__only_10_sensors.to_csv("2019_stockholm__sensor_values__daily_median__for_hours_w_at_least_75pc_uptime_per_hour_and_day__10_sensors.csv")

## *** TRY 2 : Daily median sensor values for hours with at least 75% uptime per day

In [146]:
## Daily median values 
#### based on hours with 75% coverage 
#### - only of the 10 sensors found to have sufficient coverage sensor_id_list_w_more_than_90pct_coverage

df_sensor_values__w_only_qualifying_uptime_hours_and_days__as_daily_median_value__with_nan__TRY3 = df_sensor_values__w_only_qualifying_uptime_hours_and_days__as_daily_median_value__with_nan__TRY2[ df_sensor_list_w_at_least_75pc_hourly_coverage_at_least_75_of_year.index ]
df_sensor_values__w_only_qualifying_uptime_hours_and_days__as_daily_median_value__with_nan__TRY3.head()

,7597,12129,14133,12141,16147,12393,19788,17650,18820,10823,18828,7426,22082,19778,19597,20259
2019-01-01,2.385,2.63,4.37,2.540,NaN,3.12,NaN,1.815,4.225,3.430,NaN,NaN,NaN,NaN,2.67,NaN
2019-01-02,2.400,2.33,NaN,2.610,NaN,3.09,NaN,2.200,4.330,3.800,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-03,2.100,2.20,4.20,2.800,4.3,2.90,NaN,2.050,4.415,3.415,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-04,1.430,1.60,3.13,1.485,3.0,2.05,NaN,1.300,2.970,1.930,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
df_sensor_values__w_only_qualifying_uptime_hours_and_days__as_daily_median_value__with_nan__TRY3.tail()

,7597,12129,14133,12141,16147,12393,19788,17650,18820,10823,18828,7426,22082,19778,19597,20259
2019-12-28,3.070,3.13,6.000,3.570,5.40,3.87,2.3700,2.43,5.200,NaN,4.630,5.330,6.030,NaN,NaN,3.67
2019-12-29,12.215,10.10,15.715,9.935,13.70,11.17,8.8700,9.40,14.975,NaN,19.715,15.770,16.430,12.135,NaN,NaN
2019-12-30,10.750,9.05,14.265,8.670,12.93,9.27,9.0300,8.23,13.675,NaN,12.635,9.715,12.015,12.880,NaN,NaN
2019-12-31,1.600,2.07,3.070,1.530,2.37,1.97,1.0775,1.27,3.800,NaN,1.700,2.585,2.000,2.200,NaN,NaN
2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### (UPDATED ) 
## ** Save : Daily median sensor values for hours with at least 75% uptime per day

In [142]:
df_sensor_values__w_only_qualifying_uptime_hours_and_days__as_daily_median_value__with_nan__TRY3.to_csv("2019_stockholm__sensor_values__daily_median__for_hours_w_at_least_75pc_uptime_per_hour_and_day__ALL_QUALIFYING_sensors.csv")

### PLOT sensors on map! 

In [150]:
# get the coordinates of the relevant sensors 

# - make table to put them in 
# make table to export data to 
lat_lon_of_sensors_w_sufficient_uptime_75by75uptime = pd.DataFrame( index=df_sensor_values__w_only_qualifying_uptime_hours_and_days__as_daily_median_value__with_nan__TRY2__only_10_sensors.columns )
# add lat lon columns
lat_lon_of_sensors_w_sufficient_uptime_75by75uptime['lat'] = np.nan
lat_lon_of_sensors_w_sufficient_uptime_75by75uptime['lon'] = np.nan
lat_lon_of_sensors_w_sufficient_uptime_75by75uptime

,lat,lon
7597,NaN,NaN
12129,NaN,NaN
14133,NaN,NaN
12141,NaN,NaN
16147,NaN,NaN
12393,NaN,NaN
19788,NaN,NaN
17650,NaN,NaN
18820,NaN,NaN
10823,NaN,NaN


In [151]:
# add sensor coords to the table 

for sensor_id in lat_lon_of_sensors_w_sufficient_uptime_75by75uptime.index:
    first_row_of_sensor = df_sensorData[ df_sensorData.sensor_id == sensor_id ].iloc[0]
    # set the data 
    lat_lon_of_sensors_w_sufficient_uptime_75by75uptime.loc[ sensor_id, 'lat' ] = first_row_of_sensor.lat
    lat_lon_of_sensors_w_sufficient_uptime_75by75uptime.loc[ sensor_id, 'lon' ] = first_row_of_sensor.lon
    
lat_lon_of_sensors_w_sufficient_uptime_75by75uptime

,lat,lon
7597,59.321,18.064
12129,59.341,18.040
14133,59.364,18.018
12141,59.285,18.023
16147,59.364,18.018
12393,59.343,18.033
19788,59.332,18.243
17650,59.304,18.101
18820,59.333,18.033
10823,59.333,18.033


In [152]:
# get max coordinate values 
max_coord_values = lat_lon_of_sensors_w_sufficient_uptime_75by75uptime.mean()
max_coord_values

lat    59.326000
lon    18.069812
dtype: float64

In [153]:
# set map location with the max values of map 
m = folium.Map([max_coord_values.lat, max_coord_values.lon], zoom_start=11)

In [154]:
# make markers 
for row_i in range( lat_lon_of_sensors_w_sufficient_uptime_75by75uptime.shape[0] ) : 
#     print( row_i )
    lat = lat_lon_of_sensors_w_sufficient_uptime_75by75uptime.iloc[row_i].lat
    lon = lat_lon_of_sensors_w_sufficient_uptime_75by75uptime.iloc[row_i].lon
    index = lat_lon_of_sensors_w_sufficient_uptime_75by75uptime.index[row_i]
    # print( str( index )+" : "+str( lat )+","+str( lon ) )
    percentile_val = df_annual_90th_percentile_values__based_on_daily_median_values__only_sensors_w_relevant_uptime[ index ]
    popup_text = "sensor_id : "+str( index )+" | lat : "+str( lat )+" | lon : "+str( lon )+" | 90th percentile : "+str( percentile_val )
    folium.CircleMarker( 
        location=[ lat, lon],
        radius=5,
        fill=True,
        popup=folium.Popup( popup_text, parse_html=True, max_width=200 )
        ).add_to(m)
    
m